## Up-sampling with Transposed Convolution
https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0


In [1]:
import numpy as np
import torch
import torch.nn.functional as F

In [2]:
np.__version__, torch.__version__

('1.15.4', '1.0.0.dev20181126')

### Input

In [3]:
X = torch.Tensor([[4,5,8,7],[1,8,8,8],[3,6,6,4],[6,5,7,8]])

### Weights

In [4]:
w = torch.Tensor([[1,4,1],[1,4,3],[3,3,1]])

### Convolution output - many to one => (4,4) to (2,2)

In [5]:
conv_output = F.conv2d(X[None][None],w[None][None])
conv_output

tensor([[[[122., 148.],
          [126., 134.]]]])

### Convert the filter weights to a Convolution matrix with zero padding in different places

In [6]:
conv_matrix = np.array([
               [1,4,1,0,1,4,3,0,3,3,1,0,0,0,0,0],
               [0,1,4,1,0,1,4,3,0,3,3,1,0,0,0,0],
               [0,0,0,0,1,4,1,0,1,4,3,0,3,3,1,0],
               [0,0,0,0,0,1,4,1,0,1,4,3,0,3,3,1],
              ])

### Flatten the input

In [7]:
f_X = X.view(16,1).numpy()

In [8]:
conv_matrix.shape, f_X.shape

((4, 16), (16, 1))

### Convolution using Matrix Multiplication

In [9]:
mat_mul_output = (conv_matrix @ f_X)
print(mat_mul_output.reshape(2,2))
assert np.array_equal(conv_output.numpy().squeeze(), mat_mul_output.reshape((2,2)))

[[122. 148.]
 [126. 134.]]


### Transposed Convolution using Matrix Multiplication - one to Many => (2,2) to (4,4)
#### NB: the actual weight values in the matrix does not have to come from the original convolution matrix. What’s important is that the weight layout is transposed from that of the convolution matrix.

In [10]:
(conv_matrix.T @ np.array([2,1,4,4])).reshape(4,4)

array([[ 2,  9,  6,  1],
       [ 6, 29, 30,  7],
       [10, 29, 33, 13],
       [12, 24, 16,  4]])